In [192]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/train.csv
/kaggle/input/test.csv


In [193]:
dftest=pd.read_csv('/kaggle/input/test.csv')
dftrain=pd.read_csv('/kaggle/input/train.csv')
dftraintext=pd.read_csv('/kaggle/input/train.csv',usecols=['review_by_patient'])

In [194]:
dftrain['review_by_patient']=dftrain['review_by_patient']+" "+dftrain['use_case_for_drug']+" "+dftrain['name_of_drug']

In [195]:
import string
def remove_punctuations(text):
    for punctuation in string.punctuation:
        text=text.replace(punctuation,'')
    return text

dftrain['review_by_patient']=dftrain['review_by_patient'].apply(remove_punctuations)

In [196]:
import seaborn as sns
import matplotlib.pyplot as plt
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer 
nltk.download('stopwords')
stop=set(stopwords.words('english'))
lemmatizer=WordNetLemmatizer()
def stop_stem_train(text):
    final_text=[]
    for i in text.split():
        if(i.strip().lower() not in stop):
            word=lemmatizer.lemmatize(i.strip())
            final_text.append(word)
    return " ".join(final_text)


dftrain['review_by_patient']=dftrain['review_by_patient'].apply(stop_stem_train)

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [197]:
from nltk.tokenize import sent_tokenize
sentences=dftrain['review_by_patient'].apply(lambda x: nltk.sent_tokenize(x))

In [198]:
sentences.shape

(32165,)

In [199]:
dftrain=dftrain.drop(['patient_id','name_of_drug','use_case_for_drug','review_by_patient','drug_approved_by_UIC','review_by_patient'],axis=1)

In [200]:
dftrain=pd.concat([dftrain,sentences],axis=1)

In [201]:
dftrain.head()

,effectiveness_rating,number_of_times_prescribed,base_score,review_by_patient
0,9,27,8.022969,[side effect take combination Bystolic 5 Mg Fi...
1,8,192,7.858458,[son halfway fourth week Intuniv became concer...
2,5,17,6.341969,[used take another oral contraceptive 21 pill ...
3,9,37,6.590176,[Suboxone completely turned life around feel h...
4,2,43,6.144782,[2nd day 5mg started work rock hard erection h...


In [202]:
dftrain1=dftrain

In [203]:
pip install vaderSentiment

Note: you may need to restart the kernel to use updated packages.


In [204]:
dftrain['review_by_patient'].shape

(32165,)

In [ ]:

from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
def analyze_sentiment(dftrain):
    sentiments = []
    sid = SentimentIntensityAnalyzer()
    for i in range(dftrain.shape[0]):
        line = dftrain['review_by_patient'].iloc[i]
        sentiment = sid.polarity_scores(line)
        sentiments.append([sentiment['neg'], sentiment['pos'],sentiment['neu'], sentiment['compound']])
        dftrain[['neg', 'pos', 'neu', 'compound']] = pd.DataFrame(sentiments)
    return dftrain

analyze_sentiment(dftrain)

In [ ]:
dftrain.shape[0]

In [ ]:
dftrain = dftrain.drop('review_by_patient',axis=1)

In [ ]:
dftrain

In [ ]:

my_list = []
def calculate_score(dftrain):
    for i in range(dftrain.shape[0]):
        neg = dftrain['neg'].iloc[i]
        pos = dftrain['pos'].iloc[i]
        neu = dftrain['neu'].iloc[i]
        if ((neg > pos) and (neg > neu)):
            my_list.append('0') 
        elif ((pos > neg) and (pos > neu)):
            my_list.append('1') 
        else:
            my_list.append('2') 
        
calculate_score(dftrain)       

In [ ]:
my_list

In [ ]:
dftrain['result'] = my_list

In [ ]:
dftrain

In [ ]:
dftrain.dtypes

In [ ]:
dftrain['result'] = dftrain['result'].astype(int)

In [ ]:
dftrain.dtypes

In [ ]:
dftrain

In [ ]:
dftrain = dftrain.drop(['neg','pos','neu','compound'],axis=1)

In [ ]:
dftrain

In [ ]:
dftrain

In [ ]:
X = dftrain.drop('base_score',axis=1)

In [ ]:

dftrain

In [ ]:
Y = dftrain.drop(['effectiveness_rating','number_of_times_prescribed','result'],axis=1)
Y = Y.astype(int)
Y

In [ ]:
#Y = dftrain.drop(['effectiveness_rating','number_of_times_prescribed','result'],axis=1)

In [ ]:
#X = dftrain.drop('base_score',axis=1)
X

In [ ]:
Y

In [ ]:
#dftrain["Base_score"] = pd.to_numeric(dftrain["base_score"],errors='coerce')

In [ ]:
#dftrain = dftrain.drop('base_score',axis=1)
dftrain.dtypes

In [ ]:
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
#y=Y['base_score'].values
#y = to_categorical(y)

In [ ]:
Y.dtypes

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, random_state=42, test_size=0.2)

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression


In [ ]:
clf = DecisionTreeClassifier()
regressor = RandomForestRegressor()
regressor1 = LinearRegression()  
clf.fit(X_train, y_train)

In [ ]:
y_pred = clf.predict(X_test)
y_pred

In [ ]:
#clf1= clf.fit(X_train,y_train)
clf2 = regressor.fit(X_train,y_train)

In [ ]:
#y_pred = clf2.predict(X_test)
#y_pred
np.around(y_pred, decimals=0)

In [ ]:
from sklearn import metrics 
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

In [73]:
#y_pred
#y_test

,base_score
12551,6
7640,6
24705,7
27745,6
30741,6
...,...
11180,4
16073,6
30529,6
22944,8


In [74]:
#dftest=pd.read_csv('/kaggle/input/test.csv')

In [75]:
#dftest

,patient_id,name_of_drug,review_by_patient,drug_approved_by_UIC,number_of_times_prescribed,use_case_for_drug,effectiveness_rating
0,163740,Mirtazapine,"""I&#039;ve tried a few antidepressants over th...",28-Feb-12,22,Depression,10
1,39293,Contrave,"""Contrave combines drugs that were used for al...",5-Mar-17,35,Weight Loss,9
2,208087,Zyclara,"""4 days in on first 2 weeks. Using on arms an...",3-Jul-14,13,Keratosis,4
3,23295,Methadone,"""Ive been on Methadone for over ten years and ...",18-Oct-16,21,Opiate Withdrawal,7
4,97013,Ambien,"""Ditto on rebound sleepless when discontinued....",13-Jan-15,44,Insomnia,2
...,...,...,...,...,...,...,...
10755,151266,Chantix,"""I took chantix a little over a month. It made...",11-Nov-17,2,Smoking Cessation,1
10756,139347,Armodafinil,"""This medicine kept me from sleeping the whole...",30-Sep-14,18,Narcolepsy,1
10757,159999,Tamoxifen,"""I have taken Tamoxifen for 5 years. Side effe...",13-Sep-14,43,"Breast Cancer, Prevention",10
10758,130945,Levonorgestrel,"""I&#039;m married, 34 years old and I have no ...",15-Nov-10,7,Birth Control,8


In [76]:
#dftest['review_by_patient']=dftest['review_by_patient']+" "+dftest['use_case_for_drug']+" "+dftest['name_of_drug']

In [77]:
#import string
#def remove_punctuations(text):
    #for punctuation in string.punctuation:
       # text=text.replace(punctuation,'')
    #return text

#dftest['review_by_patient']=dftest['review_by_patient'].apply(remove_punctuations)

In [78]:
#dftest

,patient_id,name_of_drug,review_by_patient,drug_approved_by_UIC,number_of_times_prescribed,use_case_for_drug,effectiveness_rating
0,163740,Mirtazapine,I039ve tried a few antidepressants over the ye...,28-Feb-12,22,Depression,10
1,39293,Contrave,Contrave combines drugs that were used for alc...,5-Mar-17,35,Weight Loss,9
2,208087,Zyclara,4 days in on first 2 weeks Using on arms and ...,3-Jul-14,13,Keratosis,4
3,23295,Methadone,Ive been on Methadone for over ten years and c...,18-Oct-16,21,Opiate Withdrawal,7
4,97013,Ambien,Ditto on rebound sleepless when discontinued I...,13-Jan-15,44,Insomnia,2
...,...,...,...,...,...,...,...
10755,151266,Chantix,I took chantix a little over a month It made m...,11-Nov-17,2,Smoking Cessation,1
10756,139347,Armodafinil,This medicine kept me from sleeping the whole ...,30-Sep-14,18,Narcolepsy,1
10757,159999,Tamoxifen,I have taken Tamoxifen for 5 years Side effect...,13-Sep-14,43,"Breast Cancer, Prevention",10
10758,130945,Levonorgestrel,I039m married 34 years old and I have no kids ...,15-Nov-10,7,Birth Control,8


In [79]:
#

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [80]:
#dftest

,patient_id,name_of_drug,review_by_patient,drug_approved_by_UIC,number_of_times_prescribed,use_case_for_drug,effectiveness_rating
0,163740,Mirtazapine,I039ve tried antidepressant year citalopram fl...,28-Feb-12,22,Depression,10
1,39293,Contrave,Contrave combine drug used alcohol smoking opi...,5-Mar-17,35,Weight Loss,9
2,208087,Zyclara,4 day first 2 week Using arm face Put vaseline...,3-Jul-14,13,Keratosis,4
3,23295,Methadone,Ive Methadone ten year currentlyI trying get d...,18-Oct-16,21,Opiate Withdrawal,7
4,97013,Ambien,Ditto rebound sleepless discontinued done stra...,13-Jan-15,44,Insomnia,2
...,...,...,...,...,...,...,...
10755,151266,Chantix,took chantix little month made psychotic split...,11-Nov-17,2,Smoking Cessation,1
10756,139347,Armodafinil,medicine kept sleeping whole 5 day body starte...,30-Sep-14,18,Narcolepsy,1
10757,159999,Tamoxifen,taken Tamoxifen 5 year Side effect severe swea...,13-Sep-14,43,"Breast Cancer, Prevention",10
10758,130945,Levonorgestrel,I039m married 34 year old kid Taking pill hass...,15-Nov-10,7,Birth Control,8


In [81]:
#from nltk.tokenize import sent_tokenize
#sentences=dftest['review_by_patient'].apply(lambda x: nltk.sent_tokenize(x))

In [82]:
#dftest = dftest.drop(['patient_id','name_of_drug','review_by_patient','drug_approved_by_UIC','use_case_for_drug'],axis=1)

In [83]:

#dftest=pd.concat([dftest,sentences],axis=1)

In [84]:
#dftest

,number_of_times_prescribed,effectiveness_rating,review_by_patient
0,22,10,[I039ve tried antidepressant year citalopram f...
1,35,9,[Contrave combine drug used alcohol smoking op...
2,13,4,[4 day first 2 week Using arm face Put vaselin...
3,21,7,[Ive Methadone ten year currentlyI trying get ...
4,44,2,[Ditto rebound sleepless discontinued done str...
...,...,...,...
10755,2,1,[took chantix little month made psychotic spli...
10756,18,1,[medicine kept sleeping whole 5 day body start...
10757,43,10,[taken Tamoxifen 5 year Side effect severe swe...
10758,7,8,[I039m married 34 year old kid Taking pill has...


In [85]:
#

,number_of_times_prescribed,effectiveness_rating,review_by_patient,neg,pos,neu,compound
0,22,10,[I039ve tried antidepressant year citalopram f...,0.206,0.254,0.540,0.4332
1,35,9,[Contrave combine drug used alcohol smoking op...,0.112,0.182,0.706,0.7334
2,13,4,[4 day first 2 week Using arm face Put vaselin...,0.099,0.180,0.721,0.3818
3,21,7,[Ive Methadone ten year currentlyI trying get ...,0.080,0.138,0.782,0.2263
4,44,2,[Ditto rebound sleepless discontinued done str...,0.189,0.000,0.811,-0.5984
...,...,...,...,...,...,...,...
10755,2,1,[took chantix little month made psychotic spli...,0.000,0.102,0.898,0.3612
10756,18,1,[medicine kept sleeping whole 5 day body start...,0.000,0.000,1.000,0.0000
10757,43,10,[taken Tamoxifen 5 year Side effect severe swe...,0.268,0.114,0.618,-0.9291
10758,7,8,[I039m married 34 year old kid Taking pill has...,0.192,0.046,0.762,-0.8817


In [86]:
#

In [87]:
#dftest['result'] = my_list

In [88]:
#dftest = dftest.drop(['neg','pos','neu','compound'],axis=1)

In [89]:
#dftest

,number_of_times_prescribed,effectiveness_rating,result
0,22,10,2
1,35,9,2
2,13,4,2
3,21,7,2
4,44,2,2
...,...,...,...
10755,2,1,2
10756,18,1,2
10757,43,10,2
10758,7,8,2


In [105]:
#Pred_test = clf.predict(dftest)

In [179]:
#pred_test1 = Pred_test - 2
#Pred_test = np.around(Pred_test, decimals=2)
#pred_test1

array([6, 6, 6, ..., 6, 3, 6])

In [180]:
#dftest.drop(['number_of_times_prescribed','effectiveness_rating','result'],axis=1)

""
0
1
2
3
4
...
10755
10756
10757
10758


In [181]:
#dftest1=pd.read_csv('/kaggle/input/test.csv')

In [182]:
#dftest1

,patient_id,name_of_drug,review_by_patient,drug_approved_by_UIC,number_of_times_prescribed,use_case_for_drug,effectiveness_rating
0,163740,Mirtazapine,"""I&#039;ve tried a few antidepressants over th...",28-Feb-12,22,Depression,10
1,39293,Contrave,"""Contrave combines drugs that were used for al...",5-Mar-17,35,Weight Loss,9
2,208087,Zyclara,"""4 days in on first 2 weeks. Using on arms an...",3-Jul-14,13,Keratosis,4
3,23295,Methadone,"""Ive been on Methadone for over ten years and ...",18-Oct-16,21,Opiate Withdrawal,7
4,97013,Ambien,"""Ditto on rebound sleepless when discontinued....",13-Jan-15,44,Insomnia,2
...,...,...,...,...,...,...,...
10755,151266,Chantix,"""I took chantix a little over a month. It made...",11-Nov-17,2,Smoking Cessation,1
10756,139347,Armodafinil,"""This medicine kept me from sleeping the whole...",30-Sep-14,18,Narcolepsy,1
10757,159999,Tamoxifen,"""I have taken Tamoxifen for 5 years. Side effe...",13-Sep-14,43,"Breast Cancer, Prevention",10
10758,130945,Levonorgestrel,"""I&#039;m married, 34 years old and I have no ...",15-Nov-10,7,Birth Control,8


In [183]:
#dftest1 = dftest1.drop(['name_of_drug','review_by_patient','drug_approved_by_UIC','number_of_times_prescribed','use_case_for_drug','effectiveness_rating'],axis=1)

In [184]:
#pred_test1.shape
#pred_test1

array([6, 6, 6, ..., 6, 3, 6])

In [185]:
#dftest1

,patient_id
0,163740
1,39293
2,208087
3,23295
4,97013
...,...
10755,151266
10756,139347
10757,159999
10758,130945


In [186]:
##mylist = []
#for i in range(Pred_test.shape[0]):
    #x = pred_test1[i]
    #mylist.append(x)

In [187]:
#mylist


[6,
 6,
 6,
 4,
 6,
 6,
 6,
 6,
 1,
 6,
 5,
 1,
 6,
 2,
 4,
 4,
 4,
 4,
 4,
 6,
 5,
 6,
 6,
 6,
 5,
 4,
 6,
 6,
 5,
 5,
 6,
 2,
 6,
 6,
 5,
 4,
 6,
 6,
 6,
 4,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 5,
 2,
 4,
 4,
 5,
 4,
 4,
 6,
 4,
 6,
 6,
 6,
 6,
 6,
 6,
 4,
 6,
 4,
 6,
 6,
 6,
 4,
 6,
 4,
 1,
 1,
 5,
 1,
 5,
 6,
 4,
 4,
 6,
 4,
 5,
 6,
 6,
 6,
 5,
 6,
 4,
 2,
 2,
 6,
 2,
 6,
 4,
 4,
 4,
 6,
 5,
 6,
 3,
 6,
 6,
 5,
 4,
 6,
 3,
 2,
 4,
 6,
 6,
 5,
 6,
 6,
 6,
 5,
 6,
 6,
 6,
 5,
 6,
 4,
 6,
 3,
 6,
 6,
 5,
 5,
 6,
 6,
 5,
 6,
 6,
 6,
 6,
 6,
 4,
 6,
 6,
 6,
 6,
 6,
 4,
 6,
 5,
 6,
 3,
 6,
 4,
 6,
 6,
 6,
 4,
 5,
 4,
 6,
 4,
 6,
 6,
 6,
 4,
 5,
 6,
 4,
 6,
 5,
 3,
 6,
 3,
 2,
 5,
 6,
 4,
 4,
 4,
 6,
 6,
 6,
 2,
 6,
 6,
 6,
 6,
 6,
 5,
 6,
 6,
 1,
 4,
 6,
 6,
 5,
 6,
 4,
 6,
 4,
 6,
 4,
 6,
 6,
 5,
 6,
 6,
 4,
 1,
 4,
 6,
 6,
 6,
 4,
 4,
 6,
 6,
 4,
 6,
 6,
 1,
 6,
 4,
 6,
 5,
 6,
 6,
 4,
 6,
 5,
 5,
 6,
 6,
 6,
 4,
 6,
 6,
 4,
 4,
 4,
 5,
 4,
 6,
 6,
 5,
 3,
 2,
 6,
 6,
 6,
 4,
 4,
 2,
 4,
 6,


In [188]:
#dftest1['base_score'] = mylist

In [189]:
#dftest1

,patient_id,base_score
0,163740,6
1,39293,6
2,208087,6
3,23295,4
4,97013,6
...,...,...
10755,151266,4
10756,139347,4
10757,159999,6
10758,130945,3


In [190]:
#dftest1.to_csv('submission8.csv', index = False)

In [191]:
#mylist

[6,
 6,
 6,
 4,
 6,
 6,
 6,
 6,
 1,
 6,
 5,
 1,
 6,
 2,
 4,
 4,
 4,
 4,
 4,
 6,
 5,
 6,
 6,
 6,
 5,
 4,
 6,
 6,
 5,
 5,
 6,
 2,
 6,
 6,
 5,
 4,
 6,
 6,
 6,
 4,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 5,
 2,
 4,
 4,
 5,
 4,
 4,
 6,
 4,
 6,
 6,
 6,
 6,
 6,
 6,
 4,
 6,
 4,
 6,
 6,
 6,
 4,
 6,
 4,
 1,
 1,
 5,
 1,
 5,
 6,
 4,
 4,
 6,
 4,
 5,
 6,
 6,
 6,
 5,
 6,
 4,
 2,
 2,
 6,
 2,
 6,
 4,
 4,
 4,
 6,
 5,
 6,
 3,
 6,
 6,
 5,
 4,
 6,
 3,
 2,
 4,
 6,
 6,
 5,
 6,
 6,
 6,
 5,
 6,
 6,
 6,
 5,
 6,
 4,
 6,
 3,
 6,
 6,
 5,
 5,
 6,
 6,
 5,
 6,
 6,
 6,
 6,
 6,
 4,
 6,
 6,
 6,
 6,
 6,
 4,
 6,
 5,
 6,
 3,
 6,
 4,
 6,
 6,
 6,
 4,
 5,
 4,
 6,
 4,
 6,
 6,
 6,
 4,
 5,
 6,
 4,
 6,
 5,
 3,
 6,
 3,
 2,
 5,
 6,
 4,
 4,
 4,
 6,
 6,
 6,
 2,
 6,
 6,
 6,
 6,
 6,
 5,
 6,
 6,
 1,
 4,
 6,
 6,
 5,
 6,
 4,
 6,
 4,
 6,
 4,
 6,
 6,
 5,
 6,
 6,
 4,
 1,
 4,
 6,
 6,
 6,
 4,
 4,
 6,
 6,
 4,
 6,
 6,
 1,
 6,
 4,
 6,
 5,
 6,
 6,
 4,
 6,
 5,
 5,
 6,
 6,
 6,
 4,
 6,
 6,
 4,
 4,
 4,
 5,
 4,
 6,
 6,
 5,
 3,
 2,
 6,
 6,
 6,
 4,
 4,
 2,
 4,
 6,
